<a href="https://colab.research.google.com/github/Micahgs/Data-science-Assignments/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
df= pd.read_csv("https://raw.githubusercontent.com/Micahgs/DATA-science-asiignment-datasets/refs/heads/main/anime.csv")

In [5]:
#Basic Exploration
df.info()
df.isnull().sum()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# Data Cleanning

In [28]:
df = df.dropna(subset=["genre", "rating"]).copy()  # Remove rows with missing genre or rating
df.loc[:, "episodes"] = df["episodes"].replace("Unknown", np.nan)
df = df.dropna(subset=["episodes"])
df["episodes"] = df["episodes"].astype(int)

# Encoding

In [29]:
df["genre"] = df["genre"].apply(lambda x: x.split(", ") if isinstance(x, str) else x)
genre_exploded = df["genre"].explode()
genre_dummies = pd.get_dummies(genre_exploded).groupby(level=0).sum()

# Normalize numerical features

In [30]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[["rating", "members", "episodes"]])
scaled_df = pd.DataFrame(scaled, columns=["rating", "members", "episodes"], index=df.index)


In [31]:
features = pd.concat([scaled_df, genre_dummies], axis=1)

# Cosine Similarity

In [33]:
cos_sim_matrix = cosine_similarity(features)

# Recommendation Function

In [36]:
def recommend_anime(title, top_n=5):
    if title not in df["name"].values:
        return f"'{title}' not found in dataset."

    idx = df[df["name"] == title].index[0]
    sim_scores = list(enumerate(cos_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    indices = [i for i, score in sim_scores]

    # Convert genre list back to string
    df["genre"] = df["genre"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

    return df.iloc[indices][["name", "genre", "type", "rating"]]

# Experement with diffrent thersholds

In [45]:
def recommend_anime(title, threshold=0.8):
    if title not in df["name"].values:
        return f"'{title}' not found in dataset."


    idx = df[df["name"] == title].index[0]


    sim_scores = list(enumerate(cos_sim_matrix[idx]))


    filtered = [(i, score) for i, score in sim_scores if i != idx and score >= threshold]

    filtered = sorted(filtered, key=lambda x: x[1], reverse=True)


    indices = [i for i, score in filtered]


    df["genre"] = df["genre"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

    return df.iloc[indices][["name", "genre", "type", "rating"]]


In [46]:
recommend_anime("Naruto", threshold=0.85)

,name,genre,type,rating
4540,Trava: Fist Planet,"Action, Comedy, Mecha, Sci-Fi, Space",OVA,6.70
1906,Mobile Suit Gundam 00 The Movie: A Wakening of...,"Action, Mecha, Sci-Fi, Space",Movie,7.41
2130,Super Robot Taisen OG: The Inspector,"Action, Mecha, Sci-Fi, Space",TV,7.35
2165,Sei Juushi Bismarck,"Action, Mecha, Sci-Fi, Space",TV,7.34
3182,Rinne no Lagrange Season 2,"Action, Comedy, Mecha, Sci-Fi",TV,7.06
3345,Rinne no Lagrange,"Action, Comedy, Mecha, Sci-Fi",TV,7.02
4601,iDOLM@STER Xenoglossia,"Action, Comedy, Mecha, Sci-Fi",TV,6.68
2828,Aldnoah.Zero 2nd Season,"Action, Mecha, Sci-Fi, Space",TV,7.15
4749,Uchuu Senshi Baldios (Movie),"Action, Mecha, Sci-Fi, Space",Movie,6.65
4776,Muteki Robo Trider G7,"Comedy, Mecha, Sci-Fi, Space",TV,6.64


In [47]:
recommend_anime("Naruto", threshold=0.8)

,name,genre,type,rating
4540,Trava: Fist Planet,"Action, Comedy, Mecha, Sci-Fi, Space",OVA,6.70
1906,Mobile Suit Gundam 00 The Movie: A Wakening of...,"Action, Mecha, Sci-Fi, Space",Movie,7.41
2130,Super Robot Taisen OG: The Inspector,"Action, Mecha, Sci-Fi, Space",TV,7.35
2165,Sei Juushi Bismarck,"Action, Mecha, Sci-Fi, Space",TV,7.34
3182,Rinne no Lagrange Season 2,"Action, Comedy, Mecha, Sci-Fi",TV,7.06
...,...,...,...,...
8440,Dennou Sentai Voogie&#039;s★Angel Gaiden: Susu...,"Action, Comedy, Mecha, Parody, Sci-Fi",OVA,5.25
8441,Dennou Sentai Voogie&#039;s★Angel: Forever and...,"Action, Comedy, Drama, Mecha, Sci-Fi",OVA,5.10
29,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,8.78
2506,Tenchi Muyou! GXP,"Action, Comedy, Harem, Mecha, Police, Sci-Fi, ...",TV,7.25


In [48]:
recommend_anime("Naruto", threshold=0.95)

,name,genre,type,rating
4540,Trava: Fist Planet,"Action, Comedy, Mecha, Sci-Fi, Space",OVA,6.7


# Evaluation:

# Interview Questions:

1. Can you explain the difference between user-based and item-based collaborative filtering?

Yes. Both are techniques used in recommendation systems, but they focus on different things.

User-based collaborative filtering tries to find users who are similar to you, based on their past behavior, like what they’ve liked or rated. Then it recommends items that those similar users liked but you haven’t seen yet. It’s like saying, "People like you also liked this."

Item-based collaborative filtering works the other way. It looks at items (like movies or anime) that are similar to each other based on how users have rated them. Then it recommends items similar to what you’ve liked before. It’s more like saying, "Because you liked this anime, here are some similar ones."

2. What is collaborative filtering, and how does it work?

Collaborative filtering is a recommendation technique that makes suggestions by learning from user behavior — not from the content of the items.

Instead of looking at what the anime is about, it looks at how people interact with it. For example, if two users have watched and rated many of the same shows, the system assumes they have similar tastes. So, if one of them watches a new anime, the other might get that as a recommendation.

There are two main types: user-based and item-based. Both use the idea that patterns in user behavior can help predict what someone might like next.

